In [1]:
#we are testing enformer at basepair resolution, let's first start with seeing if the decoder works, as that is the strange part

import torch
import sys
sys.path.append('/data1/lesliec/sarthak/caduceus')
from src.tasks.decoders import EnformerDecoder

In [2]:
d = EnformerDecoder(d_model=256, d_output=674, conjoin_train=False, bin_size=1)
d

EnformerDecoder(
  (output_transform): Linear(in_features=256, out_features=674, bias=True)
  (pool): AvgPool1d(kernel_size=(1,), stride=(1,), padding=(0,))
  (softplus): Softplus(beta=1, threshold=20)
)

In [3]:
#let's produce a random value
#batch size x seq len x d model
x = torch.randn(1, 196608, 256).cuda()

In [4]:
d = d.cuda()

In [5]:
d(x)

tensor([[[0.2687, 1.7076, 0.8778,  ..., 1.1117, 0.7774, 0.4837],
         [0.4500, 0.8824, 0.5638,  ..., 0.5702, 1.0598, 0.3637],
         [0.9785, 0.6307, 0.6277,  ..., 0.4253, 1.2073, 0.4768],
         ...,
         [0.3334, 1.3291, 0.5813,  ..., 0.3851, 0.7334, 0.6539],
         [0.8262, 1.3521, 0.6982,  ..., 0.4825, 0.7269, 0.3903],
         [0.4281, 1.1182, 0.5119,  ..., 0.7136, 1.0270, 0.9998]]],
       device='cuda:0', grad_fn=<SoftplusBackward0>)

In [2]:
#yeah so it is calculated quite quickly... hmmmmmm
#the other potential issue is the data loader, this seems to be the slowdown somehow
#I literally printed all the options

config = {
    "dataset_name": "EnformerLoader",
    "dest_path": None,
    "tokenizer_name": "char",
    "d_output": 4675,
    "rc_aug": False,
    "max_length": 196608,
    "use_padding": True,
    "max_length_val": 196608,
    "max_length_test": 196608,
    "padding_side": "left",
    "return_mask": False,
    "val_ratio": 0.0005,
    "val_split_seed": 2357,
    "add_eos": False,
    "detokenize": False,
    "val_only": False,
    "batch_size": 1,
    "batch_size_eval": 2,
    "num_workers": 0,
    "shuffle": True,
    "pin_memory": True,
    "drop_last": False,
    "fault_tolerant": False,
    "ddp": False,
    "fast_forward_epochs": None,
    "fast_forward_batches": None,
    "data_path": "/data1/lesliec/sarthak/data/borzoi/outputs/hg38/labels.zarr",
    "return_CAGE": False,
    "cell_type": "DNase",
    "kmer_len": None,
    "one_hot": True,
    "pool": 1
}

from src.dataloaders.genomics import EnformerLoader

loader = EnformerLoader(**config)

base_path: /data1/lesliec/sarthak/
dataset_name: EnformerLoader, dest_path: None, tokenizer_name: char, d_output: 4675, rc_aug: False, max_length: 196608, use_padding: True, max_length_val: 196608, max_length_test: 196608, padding_side: left, return_mask: False, val_ratio: 0.0005, val_split_seed: 2357, add_eos: False, detokenize: False, val_only: False, batch_size: 1, batch_size_eval: 2, num_workers: 0, shuffle: True, pin_memory: True, drop_last: False, fault_tolerant: False, ddp: False, fast_forward_epochs: None, fast_forward_batches: None, data_path: /data1/lesliec/sarthak/data/borzoi/outputs/hg38/labels.zarr, return_CAGE: False, cell_type: DNase, kmer_len: None, one_hot: True, pool: 1


In [3]:
loader.setup()

**Using Char-level tokenizer**


In [4]:
dataset = loader.dataset_train

In [5]:
out = dataset[0]

In [6]:
print(out[0].shape, out[1].shape)

torch.Size([4, 196608]) torch.Size([114688, 674])


In [7]:
for i in range(10):
    out = dataset[i]
    print(out[0].shape, out[1].shape)
#takes a few seconds for each, not ideal but reasonable...

torch.Size([4, 196608]) torch.Size([114688, 674])
torch.Size([4, 196608]) torch.Size([114688, 674])
torch.Size([4, 196608]) torch.Size([114688, 674])
torch.Size([4, 196608]) torch.Size([114688, 674])
torch.Size([4, 196608]) torch.Size([114688, 674])
torch.Size([4, 196608]) torch.Size([114688, 674])
torch.Size([4, 196608]) torch.Size([114688, 674])
torch.Size([4, 196608]) torch.Size([114688, 674])
torch.Size([4, 196608]) torch.Size([114688, 674])
torch.Size([4, 196608]) torch.Size([114688, 674])


In [9]:
#now lets iterate through the data loader itself
true_loader = loader._data_loader(loader.dataset_train, batch_size=loader.batch_size,
                                   shuffle=loader.shuffle, sampler=None)

In [10]:
#now go through this loader
for i, data in enumerate(true_loader):
    print(i)
    if i > 10:
        break

0
1
2
3
4
5
6
7
8
9
10
11


In [12]:
#see this is relatively quick?!?
len(data)

2

In [13]:
loader.batch_size

1

In [14]:
data

[tensor([[[0., 1., 0.,  ..., 0., 0., 0.],
          [1., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 1.,  ..., 1., 1., 1.]]]),
 tensor([[[0.0800, 0.0400, 0.0000,  ..., 0.0000, 0.2600, 0.0300],
          [0.0800, 0.0400, 0.0000,  ..., 0.0000, 0.2600, 0.0300],
          [0.0800, 0.0400, 0.0000,  ..., 0.0000, 0.2600, 0.0300],
          ...,
          [0.1100, 0.0300, 0.0000,  ..., 0.0700, 0.0000, 0.0800],
          [0.1100, 0.0300, 0.0000,  ..., 0.0700, 0.0000, 0.0800],
          [0.1100, 0.0300, 0.0000,  ..., 0.0700, 0.0000, 0.0800]]])]

In [15]:
data[0].shape

torch.Size([1, 4, 196608])

In [16]:
data[1].shape

torch.Size([1, 114688, 674])

In [17]:
#so why does this work but it doesn't in the training script?!?
#oh so it runs this command
true_loader = loader.train_dataloader()

In [18]:
for i, data in enumerate(true_loader):
    print(i)
    if i > 10:
        break

0
1
2
3
4
5
6
7
8
9
10
11


In [19]:
data[0].shape

torch.Size([1, 4, 196608])

In [20]:
data[1].shape

torch.Size([1, 114688, 674])

In [21]:
#huh so this works too!! That's literally whaat was slow tho??
true_loader = loader.train_dataloader()
next(iter(true_loader))

[tensor([[[0., 0., 0.,  ..., 0., 1., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [1., 1., 1.,  ..., 1., 0., 1.]]]),
 tensor([[[0.0100, 0.0400, 0.0000,  ..., 0.0000, 0.0500, 0.0200],
          [0.0100, 0.0400, 0.0000,  ..., 0.0000, 0.0500, 0.0200],
          [0.0100, 0.0400, 0.0000,  ..., 0.0000, 0.0500, 0.0200],
          ...,
          [0.0800, 0.1200, 0.1899,  ..., 0.0700, 0.0000, 0.1200],
          [0.0800, 0.1200, 0.1899,  ..., 0.0700, 0.0000, 0.1200],
          [0.0800, 0.1200, 0.1899,  ..., 0.0700, 0.0000, 0.1200]]])]

In [23]:
#so it seems like we have an issue with the loader still??
hparams = {'num_workers': 1, 'pin_memory': True, 'drop_last': True}
true_loader = loader.train_dataloader(**hparams)

In [24]:
for i, data in enumerate(true_loader):
    print(i)
    if i > 10:
        break

KeyboardInterrupt: 

In [25]:
#ok there we go so it's something based on this?
true_loader = loader.train_dataloader()
for i, data in enumerate(true_loader):
    print(i)
    if i > 10:
        break

0
1
2
3
4
5
6
7
8
9
10
11


In [26]:
#ok so we narroweddown the problem. Let's modify the h params, is it num workers, is it the pinning or the dropping?

hparams = {'num_workers': 0, 'pin_memory': True, 'drop_last': True}
true_loader = loader.train_dataloader(**hparams)
for i, data in enumerate(true_loader):
    print(i)
    if i > 10:
        break

0
1
2
3
4
5
6
7
8
9
10
11


In [27]:
#ok so it has to do with the number of workers!!
hparams = {'num_workers': 1, 'pin_memory': True, 'drop_last': True}
true_loader = loader.train_dataloader(**hparams)
for i, data in enumerate(true_loader):
    print(i)
    if i > 10:
        break

KeyboardInterrupt: 

# test pooling

In [1]:
#we need to test the pooling


import torch
import sys
sys.path.append('/data1/lesliec/sarthak/caduceus')
from src.tasks.decoders import EnformerDecoder
d = EnformerDecoder(d_model=256, d_output=674, conjoin_train=False, bin_size=128)
d

EnformerDecoder(
  (output_transform): Linear(in_features=256, out_features=674, bias=True)
  (pool): AvgPool1d(kernel_size=(128,), stride=(128,), padding=(0,))
  (softplus): Softplus(beta=1, threshold=20)
)

In [2]:
x = torch.randn(1, 196608, 256)

In [3]:
o = d(x)

In [4]:
o.shape

torch.Size([1, 896, 674])

In [ ]:
#right that looks good! But it doesn't work?